# Let RNN(GRU) write a poem(海子的诗)

在[Gluon教程](http://zh.gluon.ai/chapter_recurrent-neural-networks/rnn-scratch.html)中介绍了如何使用rnn写周杰伦style的词，我觉得很有趣，试着使用这种方式写海子的诗歌。

## 1 文本的处理

In [1]:
with open('./data/haizi.txt') as f:
    corpus_chars = f.read()

corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:20000]

idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
corpus_indices = [char_to_idx[char] for char in corpus_chars]

vocab_size = len(char_to_idx)
print('vocab size:', vocab_size)

vocab size: 1649


In [2]:
#使用one-hot encoding  
def get_inputs(data):
    return [nd.one_hot(X, vocab_size) for X in data.T]

 原始的RNN有个重要的问题就是，随着序列的前进，模型会渐渐‘忘掉’时间上比较久远的信息，为了保留较为久远的信息，一般会采用门控方法，具体包括GRU和LSTM，这里我们使用GRU。

## 2 定义参数
   和一般的RNN唯一的不同带门控的RNN，在定义参数时会多定义一个重置们和更新门。

In [3]:
#初始化模型参数
import mxnet as mx

# 尝试使用GPU
from mxnet import nd
import utils
ctx = utils.try_gpu()
print('Will use', ctx)

input_dim = vocab_size
# 隐含状态长度
hidden_dim = 256
output_dim = vocab_size
std = .01

def get_params():
    # 隐含层
    W_xz = nd.random_normal(scale=std, shape=(input_dim, hidden_dim), ctx=ctx)  #更新
    W_hz = nd.random_normal(scale=std, shape=(hidden_dim, hidden_dim), ctx=ctx)
    b_z = nd.zeros(hidden_dim, ctx=ctx)

    W_xr = nd.random_normal(scale=std, shape=(input_dim, hidden_dim), ctx=ctx) #重置
    W_hr = nd.random_normal(scale=std, shape=(hidden_dim, hidden_dim), ctx=ctx)
    b_r = nd.zeros(hidden_dim, ctx=ctx)

    W_xh = nd.random_normal(scale=std, shape=(input_dim, hidden_dim), ctx=ctx)   #候选隐含层，在naive rnn的基础上多乘了个Reset unit
    W_hh = nd.random_normal(scale=std, shape=(hidden_dim, hidden_dim), ctx=ctx)
    b_h = nd.zeros(hidden_dim, ctx=ctx)

    # 输出层
    W_hy = nd.random_normal(scale=std, shape=(hidden_dim, output_dim), ctx=ctx)
    b_y = nd.zeros(output_dim, ctx=ctx)

    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hy, b_y]
    for param in params:
        param.attach_grad()
    return params

Will use cpu(0)


In [4]:
def gru_rnn(inputs, H, *params):
    # inputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵
    # H: 尺寸为 batch_size * hidden_dim 矩阵
    # outputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hy, b_y = params
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + R * nd.dot(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = nd.dot(H, W_hy) + b_y
        outputs.append(Y)
    return (outputs, H)

## 3 具体训练

具体的训练方法（train_and_predict_rnn）放在utils.py下。
在实际写诗的时候(实际就是预测下一个词,然后不断迭代），需要定义一下开头。

In [5]:
seq1 = '我'
seq2 = '你'
seq3 = '雨夜'
seqs = [seq1, seq2, seq3]

utils.train_and_predict_rnn(rnn=gru_rnn, is_random_iter=False, epochs=200,
                            num_steps=35, hidden_dim=hidden_dim,
                            learning_rate=0.2, clipping_norm=5,
                            batch_size=32, pred_period=20, pred_len=100,
                            seqs=seqs, get_params=get_params,
                            get_inputs=get_inputs, ctx=ctx,
                            corpus_indices=corpus_indices,
                            idx_to_char=idx_to_char, char_to_idx=char_to_idx)

Epoch 20. Training perplexity 272.715964
 -  我的  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  
 -  你 我的人的  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我
 -  雨夜 我的人的  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我的子  我是我

Epoch 40. Training perplexity 147.534489
 -  我    【【铜】  我们在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头
 -  你    【【铜】  我们在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头
 -  雨夜   【【我的村庄 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在我的头 在

Epoch 60. Training perplexity 77.690218
 -  我    【我的诗人    【我的手     【死亡】  我们在地上 在沙漠上  我们在地上 在赤道上  我们在地上 在赤道上  我们在地上 在赤道上  我们在地上 在赤道上  我们在地上 在赤道上  我
 -  你    【我的诗人    【我的手     【死亡】  我们在地上 在沙漠上  我们在地上 在赤道上  我们在地上 在赤道上  我们在地上 在赤道上  我们在地上 在赤道上  我们在地上 在赤道上  我
 -  雨夜 我的头骨 我是一个人 